In [2]:
import pymesh 
import pymeshlab
import numpy as np

In [1]:
import pymeshlab


def example_load_mesh(Name):
   

    # create a new MeshSet
    ms = pymeshlab.MeshSet()

    # load a new mesh in the MeshSet, and sets it as current mesh
    # the path of the mesh can be absolute or relative
    ms.load_new_mesh(Name)

    print(len(ms))  # now ms contains 1 mesh
    # instead of len(ms) you can also use:
    print(ms.number_meshes())

    # load a new mesh, and sets it as current mesh
    
    # set the first mesh (id 0) as current mesh
    ms.set_current_mesh(0)

    # print the number of vertices of the current mesh
    print(ms.current_mesh().vertex_number())

    assert ms.current_mesh().vertex_number() == 19985

    
  

In [3]:
def calculateAngle(A, B, C):
    v1 = A - B
    v2 = C - B
    theta_rad = np.arccos(np.dot(v1, v2)/(np.linalg.norm(v1) * np.linalg.norm(v2)))

    return theta_rad

def area(P, Q, R):
    A = Q - P
    B = R - P
    C = np.cross(A, B)
    return 0.5*np.linalg.norm(C)


def voronoi_area(P, Q, R):
    PR =np.linalg.norm(P - R)
    PQ = np.linalg.norm(P - Q)
    
    angleQ = calculateAngle(P, Q, R)
    angleR = calculateAngle(P, R, Q)

    if(angleQ < np.pi/2 and angleR < np.pi/2 and angleQ + angleR > np.pi/2):
        cotQ = 1.0/np.tan(angleQ)
        cotR = 1.0/np.tan(angleR)
        return (PR*PR*cotQ + PQ*PQ*cotR)/8
    elif(angleQ + angleR < np.pi/2):
        return area(P, Q, R)/2
    else:
        return area(P, Q, R)/4


def areas_local_mixed_Voronoi_cell(mesh):
    vert_matrix = mesh.vertex_matrix()
    voronoi_areas = np.zeros(vert_matrix.shape[0])

    face_matrix = mesh.face_matrix()
    for face in face_matrix:
        voronoi_areas[face[0]] += voronoi_area(vert_matrix[face[0]], vert_matrix[face[1]], vert_matrix[face[2]])
        voronoi_areas[face[1]] += voronoi_area(vert_matrix[face[1]], vert_matrix[face[0]], vert_matrix[face[2]])
        voronoi_areas[face[2]] += voronoi_area(vert_matrix[face[2]], vert_matrix[face[1]], vert_matrix[face[0]])

    return voronoi_areas

In [5]:
ms = pymeshlab.MeshSet()
ms.load_new_mesh("face.obj")
ms.set_current_mesh(0)
m = ms.current_mesh()
face_matrix = m.face_matrix()

def faces_of_(v,matriz):
    mascara = np.any(matriz == v, axis=1)

    # Obtener los tripletes que contienen 'x' utilizando la máscara
    tripletes_con_x = matriz[mascara]
    
    return tripletes_con_x

def vertices_incidentes(v,vector):
    mascara = (vector != v)
    valores_diferentes_a_v = vector[mascara]
    return valores_diferentes_a_v



print(faces_of_(1114,face_matrix))

for faces in faces_of_(1114,face_matrix):
    print(vertices_incidentes(1114,faces)[0],vertices_incidentes(1114,faces)[1])



[[1102 1107 1114]
 [1102 1114 1113]
 [1107 1115 1114]
 [1113 1114 1092]
 [1114 1115 1095]
 [1114 1095 1092]]
1102 1107
1102 1113
1107 1115
1113 1092
1115 1095
1095 1092


In [43]:

        


def gaussian_curvature(mesh):
    
    #Calculamos el area locales mezcladas de voronoi para cada vertice de "mesh"
    areas = areas_local_mixed_Voronoi_cell(mesh)
    face_matrix = mesh.face_matrix() #matrices de caras y vertices
    vert_matrix = mesh.vertex_matrix()
    res = np.zeros(vert_matrix.shape[0]) #vector donde guardaremos las val de Curvaturas 
    
    #Cuantos vertices hay en la malla esta dado por num_vertices
    num_vertices = vert_matrix.shape[0]

    for vertice  in range(num_vertices):
        theta_sum = 0
        for face in faces_of_(vertice,face_matrix):
            A = vert_matrix[vertices_incidentes(vertice,face)[0]]
            B = vert_matrix[vertice]
            C = vert_matrix[vertices_incidentes(vertice,face)[1]]
            theta_sum += calculateAngle(A,B,C)
        
        res[vertice] = (1/abs(areas[vertice])) * (np.pi*2 - theta_sum) 
    
    return res

    
     
def gaussian_curvature2(mesh):
    
    #Calculamos el area locales mezcladas de voronoi para cada vertice de "mesh"
    face_matrix = mesh.face_matrix() #matrices de caras y vertices
    vert_matrix = mesh.vertex_matrix()
    res = np.zeros(vert_matrix.shape[0]) #vector donde guardaremos las val de Curvaturas 
    
    #Cuantos vertices hay en la malla esta dado por num_vertices
    num_vertices = vert_matrix.shape[0]

    for vertice  in range(num_vertices):
        theta_sum = 0
        for face in faces_of_(vertice,face_matrix):
            A = vert_matrix[vertices_incidentes(vertice,face)[0]]
            B = vert_matrix[vertice]
            C = vert_matrix[vertices_incidentes(vertice,face)[1]]
            theta_sum += calculateAngle(A,B,C)
        
        res[vertice] =  np.pi*2 - theta_sum
    
    return res

ms = pymeshlab.MeshSet()

    # load a new mesh in the MeshSet, and sets it as current mesh
    # the path of the mesh can be absolute or relative
ms.load_new_mesh("face.obj")
ms.set_current_mesh(0)

    
curvaturas_gaussianas = gaussian_curvature(ms.current_mesh())

print(curvaturas_gaussianas)


[-4.87024861e-02 -2.20709072e-01 -7.26757949e-01 ...  1.49410497e+02
  7.96216448e+02  1.30349015e+03]


In [41]:


def colorize(T):
    colors = np.zeros((len(T), 4))
    Tmin = np.min(T)
    Tmax = np.max(T)

    for i in range(len(T)):
        if T[i] <= Tmin/500.0:
            colors[i] = [0.0, 0.0, 1.0, 1.0]
        elif (T[i] > Tmin/500.0 and T[i] <= 0):
            colors[i] = [1.0 - T[i]*(500/Tmin), 1.0 - T[i]*(500/Tmin), 1.0, 1.0]
        elif (T[i] < Tmax/500.0 and T[i] > 0):
            colors[i] = [1.0, 1.0 - T[i]*(500/Tmax), 1.0 - T[i]*(500/Tmax), 1.0]
        else:
            colors[i] = [1.0, 0.0, 0.0, 1.0]
    
    return colors

# Example usage:
T = curvaturas_gaussianas
colores = colorize(T)
print(colores.shape[1])


4


False


In [42]:
x=ms.current_mesh()
m1 = pymeshlab.Mesh(vertex_matrix = x.vertex_matrix(),
                    face_matrix   = x.face_matrix(),
                    v_color_matrix = colores)

ms.add_mesh(m1)
ms.save_current_mesh("gaussian_curvature_face.obj")

In [52]:
seats = pymeshlab.MeshSet()

for i in range(1, 400):
    numero_archivo = str(i).zfill(4)  # Formatear el número como "0001", "0002", etc.
    nombre_archivo = f"{numero_archivo}.obj"
    try:
        # Intenta procesar el archivo
        # Si el archivo no existe, generará una excepción FileNotFoundError
        with open("assignment2/seats/"+nombre_archivo, 'r') as archivo:
            # Realiza aquí las operaciones que desees con el archivo
            seats.load_new_mesh("assignment2/seats/"+nombre_archivo)
    except FileNotFoundError:
        # El archivo no existe, simplemente pasa a la siguiente iteración
        continue

real_num_seats = len(seats)
print(len(seats))


334


In [62]:
X_M = np.zeros(real_num_seats)
for i in range(real_num_seats):
    seats.set_current_mesh(i)
    Curvatura_Gaussiana_seat = gaussian_curvature2(seats.current_mesh())
    X_M[i] = round(np.sum(Curvatura_Gaussiana_seat)/(2*np.pi) )


In [63]:
#print(X_M)
def contar_repeticiones(vector):
    # Crear un diccionario para almacenar las cuentas de cada valor
    cuentas = {}
    
    # Contar las repeticiones de cada valor en el vector
    for valor in vector:
        if valor in cuentas:
            cuentas[valor] += 1
        else:
            cuentas[valor] = 1
    
    # Imprimir los resultados
    for valor, count in cuentas.items():
        print(f"Valor: {valor}, Count: {count}")

contar_repeticiones(X_M)

Valor: 2.0, Count: 77
Valor: -2.0, Count: 29
Valor: -6.0, Count: 58
Valor: 0.0, Count: 121
Valor: -8.0, Count: 15
Valor: -12.0, Count: 4
Valor: -14.0, Count: 5
Valor: -10.0, Count: 6
Valor: -4.0, Count: 19


In [64]:
print(X_M)

[  2.  -2.  -6.  -6.   0.  -8.   2.   0.  -6.   2.   0.   2.   0.   2.
  -6.   0. -12.  -6.   2.   0.  -8.  -6.   0. -14. -14.   2.   2.   0.
   0.   0.  -2.   0.   0.  -6.  -8.   0.   0.   0.  -8.   0.   2.   0.
   2.   2.   0.   0.   0.  -6.  -8.  -6.  -6.   0.   0.   0.   0.   0.
   2.   0. -10.   0.  -4.   0.   0.   0.   0.   2.   0.   0.   0.   0.
   2.   0.   0.   2.   0.   2.   0. -10.   0.   2.  -6.  -4.  -6.   0.
   0.  -6.   2.   0.   0.  -2.  -2.  -4.   0.  -2.  -2.  -6.   0.   2.
   2.   2.   0.   2.   2.   0.   2. -12.   2.  -2.  -2.  -2.  -6.  -2.
  -6.  -6.  -6.  -6. -14.   0.   0.   0.  -6.  -6.   0.   2.   2.   0.
  -4.   0.   0.  -2.  -4.  -4.   0.  -2.  -2.  -6.  -8.   0.   0.  -8.
   2.   0.   0.   2.   2.   0.   0.   2.   2.   0. -12.  -6.  -8.   2.
  -2.  -2.  -6.  -6. -10.  -6.   0.   0.   0.  -6.  -6.   2.   2.  -4.
   0.   0.  -2.  -4.  -4.  -4.   0.  -2.  -6.   0.  -8.   0.   0.   0.
  -8.   0.   0.   2.   2.  -8.   0.   2.   0.   2.   2.   2.   2.   0.
  -6. 